## __Download Spectrum Data by Obsid__ ##

* Using threading module can accelerate downloading the spectrum fits a lot  
__But there may still be some warnings__

In [1]:
# necessary module
import urllib as url
from urllib.parse import urlparse

import threading

import pandas as pd
import re
import os
from tqdm import tqdm, trange

In [2]:
cross_match_file = r'C:\Users\Administrator\Desktop\大创\Data\temp\selected_data_None_None_cross_match.csv'
source = pd.read_csv(cross_match_file ,sep = '|' ,index_col = ['combined_obsid'])
obsid = list(source.T)
source = pd.read_csv(cross_match_file ,sep = '|' ,index_col = ['inputobjs_input_id'])
input_id = list(source.T)

In [3]:
source_file = r'C:\Users\Administrator\Desktop\大创\Data\temp\selected_data_None_None.csv'
source = pd.read_csv(source_file ,index_col = ['EDJ_name'])
name = list(source.T)
source = None
match_name = []
for i in trange(0 ,len(input_id)):
    match_name.append(name[input_id[i]])

100%|██████████| 9270/9270 [00:00<00:00, 1159352.30it/s]


In [4]:
def file_load_in(file_name , save_path , data):
    try:
        save_file = save_path +'/'+ file_name 
        with open(save_file, 'wb+') as fh:
            fh.write(data)
            fh.close()
    except FileExistsError as e:
        print(file_name)
        
def download(link , save_path , matchs = None):
    #pool_sema.acquire() 
    for attempt in range(0,6):
        try:
            response = url.request.urlopen(link, timeout = 0.5)
            file_name = response.getheader("Content-disposition").split('=')[1]
            data = response.read()
        except Exception as e:
            if attempt == 5:
                print('unsuccess to download -- ' + link)
                print(save_path)
        else: 
            if isinstance(matchs , str):
                save_path_folder = save_path + '/' + matchs
                query = url.parse.urlparse(link).path
                save_path_folder = save_path_folder + '/' + str(re.findall("\d+",query)[3])
            else:
                query = url.parse.urlparse(link).path
                save_path_folder = save_path + '/' + str(re.findall("\d+",query)[3])
                
            if not os.path.exists(save_path_folder):
                os.makedirs(save_path_folder)
            file_load_in(file_name , save_path_folder , data)
            break
    #pool_sema.release()

In [5]:
urls = []
for i in trange(0,len(obsid)):
    link='http://www.lamost.org:80/dr8/v2.0/spectrum/fits/{0}?token='.format(obsid[i])
    urls.append(link)

100%|██████████| 9270/9270 [00:00<00:00, 371035.66it/s]


In [6]:
threads = []
for i in trange(len(urls)):
    t = threading.Thread(target = download, args=(urls[i] ,r'C:\Users\Administrator\Desktop\大创\Data\temp\test1' ,match_name[i]))
    threads.append(t)

100%|██████████| 9270/9270 [00:02<00:00, 4486.73it/s]


In [6]:
from concurrent.futures import ThreadPoolExecutor,as_completed
if __name__ == '__main__':
    e = []
    threads = []
    executor = ThreadPoolExecutor(max_workers=2)
    for i in trange(len(urls)):
        t = executor.submit(download, urls[i] ,r'C:\Users\Administrator\Desktop\大创\Data\temp\test1' ,match_name[i])
        threads.append(t)
    for i in as_completed(threads):
        e.append(i.result())
    executor.shutdown()


100%|██████████| 9270/9270 [00:00<00:00, 19903.23it/s]


unsuccess to download
unsuccess to download
unsuccess to download
